# Cleaning Data

The data sets that we have come with a lot of stats about the players. In this step we are going through and removing players that didn't come from the NCAA (Europe) and trimming down the attributes to the things that we believe are important.

In [18]:
from tabulate import tabulate
import operator
import random

def get_table(filename):
    table = []
    infile = open(filename)
    lines = infile.readlines()
    for line in lines:
        add_it = True
        line = line.strip()
        line = line.strip('\n')
        values = line.split(",")
        for val in values:
            #looking for players that don't have 
            # sufficient stats.
            if val == 'NA':
                add_it = False
        if add_it:
            #TODO choose the values to add
            to_add = []
            to_add.append(values[6:])
            table.append(to_add)
    infile.close()
    return table

def main():
    table = get_table('datasets/firstRoundPicks_withCollegeStats.csv')
   
    f = open('datasets/first_rounds_cleaned.csv', 'w')
    
    #just using tabulate because it makes it nice to see,
    #change later
    f.write(tabulate(table))
    f.close()
    
    

if __name__ == '__main__':
    main()

# Basic KNN_functions

In [19]:
def compute_distances_categorical(v1,v2):
    dist = 0
    for i in range(len(v1)):
        if v1[i] != v2[i]:
            dist += 1
    return dist

def get_frequencies(table, column_index):
    '''
    get values and counts given a table and 
    the column_index
    '''
    column = sorted(get_column(table, column_index))
    values = []
    counts = []
    for value in column:
        if value not in values:
            values.append(value)
            counts.append(1)
        else:
            counts[-1] += 1
    return values, counts

def get_column(table, column_index):
    column = []
    for row in table:
        if row[column_index] != 'NA':
            column.append(row[column_index])
    return column

In [20]:
def knn_guess(train_set, test_set, k_val):
    '''
    classifier using knn given a test set, train set and k value
    '''
    
    init_len = len(train_set)
    
    right = 0
    wrong = 0

    for row in train_set:
        row.append(compute_distances_categorical(row, test_set))  

    k = k_val
    
    length_of_rows = len(train_set[0])
    train_set.sort(key=operator.itemgetter(length_of_rows-1))

    top_k = train_set[:k]

    # now use get_frequencies to get the counts of each label for majority voting
    values , counts = get_frequencies(train_set[:k],3)

    maximum = counts[0]
    our_guess = values[0]
    for i in range(len(values)):
        if counts[i] > maximum:
            our_guess = values[i]

    for row in train_set:
        while (len(row) > init_len):
            row.pop()
    
    #get the real answer
    return our_guess

In [21]:
def k_fold(table):
    randomized = table[:]
    n = len(randomized)

    for i in range(n):
        rand_index = random.randrange(0,n)
        randomized[i], randomized[rand_index] = randomized[rand_index], randomized[i]


    folds = [[] for i in range(10)]
    x = 0
    for i in range(len(randomized)):
        if x > 9:
            x = 0
        folds[x].append(randomized[i])
        x += 1
    return folds

In [22]:
def knn_matrix(table):    
    folds = k_fold(table)
    print("---------------------")
    print("Self-Coded: KNN")
    print("---------------------")
    confus = [[] for i in range(2)]
    total = 0
    wrong = 0
    right = 0
    for row in confus:
        i = 0
        while i < 2:
            row.append(0)
            i += 1

    for i in range(len(folds)):
        train_set = []
        for x in folds:
            if x != folds[i]:
                for item in x:
                    train_set.append(item)
        for j in folds[i]:
            my_guess = knn_guess(train_set, j, 10)
            if my_guess == j[3]:
                right += 1
            else:
                wrong += 1
            if j[3] == "yes":
                if my_guess == "yes":
                    confus[0][0] += 1
                else:
                    confus[0][1] += 1
            else:
                if my_guess == "yes":
                    confus[1][0] += 1
                else:
                    confus[1][1] += 1
            total += 1
    print("KNN: accuracy %.2f , error rate = %.2f" % ((right/total) , (wrong/total)))
   
    print("\n----------------------------")
    print("Confusion matrix: KNN")
    print("----------------------------")
    #create the confusion matrix

    i = 0
    for row in confus:
        tot = 0
        #insert the total
        for j in range(len(row)):
            tot += row[j]
            
            if i == j:
                correct = row[j]

        row.append(tot)


        #insert the total percent at the end
        if tot != 0:
            row.append("%.7f" % (correct/tot))
        else:
            row.append("NA")

        i += 1
    confus[0].insert(0,"yes")
    confus[1].insert(0,"no")

    heads = ['yes','no','total','recognition %']
   

    print(tabulate(confus, headers = heads))

In [23]:
#fxn that calls KNN-self-done
table = get_table('datasets/firstRoundPicks_withCollegeStats.csv')
    
knn_matrix(table)

---------------------
Self-Coded: KNN
---------------------


IndexError: list index out of range